In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as st
import seaborn as sns
import pandas_profiling

chromo = pd.read_excel('GSE114559_SuppTable7-Gene-Expression-Table.xlsx')

#remove rows with zeros
clean_chromo = chromo[chromo!=0].dropna()

#remove duplicate genes
clean_chromo = clean_chromo.drop_duplicates(subset ="gene", 
                     keep = False, inplace = False) 

#transpose the dataset
clean_transposed = clean_chromo.T

#add new index and remove old transposed index
clean_transposed.reset_index(inplace=True)
clean_transposed.drop("index",axis=1,inplace=True)

clean_transposed.columns = clean_transposed.iloc[0]

#create copy with gene descriptions on it
clean_transposed_descriptions = clean_transposed[0:2]

#labels scraped from https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi 
labels = ["Normal",
"Normal",
"Normal",
"Normal",
"Normal",
"Trisomy",
"Trisomy",
"Trisomy",
"Trisomy",
"Trisomy",
"Trisomy",
"Trisomy",
"Trisomy",
"Trisomy",
"Trisomy",
"Trisomy",
"Trisomy",
"Trisomy",
"Trisomy",
"Trisomy",
"Trisomy",
"Trisomy",
"Trisomy",
"Trisomy",
"Trisomy",
"Trisomy",
"Trisomy",
"Trisomy",
"Trisomy",
"Trisomy",
"Trisomy",
"Trisomy",
"Trisomy",
"Trisomy",
"Trisomy",
"Trisomy",
"Trisomy",
"Trisomy",
"Trisomy",
"Trisomy",
"Trisomy",
"Trisomy",
"Trisomy",
"Trisomy",
"Trisomy",
"Trisomy",
"Trisomy",
"Trisomy",
"Trisomy",
"Trisomy",
"Trisomy",
"Trisomy",
"Trisomy",
"Trisomy",
"Trisomy",
"Trisomy",
"Trisomy",
"Trisomy",
"Monosomy",
"Monosomy",
"Monosomy",
"Monosomy",
"Monosomy",
"Monosomy",
"Monosomy",
"Monosomy",
"Monosomy",
"Monosomy",
"Monosomy",
"Monosomy",
"Monosomy",
"Monosomy",
"Monosomy",
"Monosomy",
"Monosomy",
"Monosomy",
"Monosomy",
"Monosomy",
"Monosomy",
"Monosomy",
"Monosomy",
"Monosomy",
"Monosomy",
"Monosomy",
"Monosomy",
"Monosomy",
"Monosomy",
"Monosomy",
"Monosomy",
"Monosomy",
"Monosomy",
"Monosomy",
"Monosomy",
"Monosomy",
"Monosomy",
"Monosomy",
"Monosomy",
"Monosomy",
"Monosomy",
"Monosomy",
"Monosomy"]


#remove rows 0 & 1 and add class to the dataset

clean_transposed = clean_transposed.drop([clean_transposed.index[0] , clean_transposed.index[1]])
clean_transposed['class'] = labels

#keep record of all classes before working with just normal and trisomy 
clean_transposed_all = clean_transposed

#Change class labels to 
clean_transposed_all = clean_transposed_all.replace('Normal','NonTrisomy')
clean_transposed_all = clean_transposed_all.replace('Monosomy','NonTrisomy')

clean_transposed_all['class'].value_counts()

#Bring in the modelling dependancies
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier

#Split the new dataset and create new training and test data
features = clean_transposed_all.iloc[:,:-1]
result = clean_transposed_all.iloc[:,-1]
X_train, X_test, y_train, y_test = train_test_split(features, result, test_size = 0.2, random_state = 14)

#random forest first
rf = RandomForestClassifier()
rf.fit(X_train, y_train)
# Making predictions on unseen data
predictions_rf = rf.predict(X_test)

#test random forest validity
print(classification_report(y_test, predictions_rf))
print(confusion_matrix(y_test, predictions_rf))

#pick out the best features
a =list(zip(features, rf.feature_importances_))

#build list of features to use in model

columns_to_use = []
for list in a:
    if list[1]>0:
        print(list)
        columns_to_use.append(list[0])

#now produce a dataset from the useful columns
reduced_clean_transposed_all = clean_transposed[columns_to_use]
labels = clean_transposed_all['class']
reduced_clean_transposed_all = reduced_clean_transposed_all.join(labels)

#Split reduced dataset for logistic regression
Logfeatures = reduced_clean_transposed_all.iloc[:,:-1]
Logresult = reduced_clean_transposed_all.iloc[:,-1]
X_train, X_test, y_train, y_test = train_test_split(Logfeatures, Logresult, test_size = 0.2, random_state = 14)

#normalise the data
scaler = StandardScaler()
X_train_std = scaler.fit_transform(X_train)
X_test_std = scaler.fit_transform(X_test)

#Now produce a logistic regression classifyer for Trisomy vs. NonTrisomy

logmodel = LogisticRegression(solver='liblinear', class_weight='balanced')
logmodel.fit(X_train_std, y_train)
predictions = logmodel.predict(X_test_std)

print(classification_report(y_test, predictions))
print(confusion_matrix(y_test, predictions))



              precision    recall  f1-score   support

  NonTrisomy       0.50      0.67      0.57         9
     Trisomy       0.67      0.50      0.57        12

    accuracy                           0.57        21
   macro avg       0.58      0.58      0.57        21
weighted avg       0.60      0.57      0.57        21

[[6 3]
 [6 6]]
('ANXA3', 0.004658746797111578)
('ANXA5', 0.006390977443609026)
('B9D2', 0.008955354409899866)
('BICD1', 0.005071204786405924)
('C1orf109', 0.01686812771914336)
('CACNA2D2', 0.004810698994563914)
('CACUL1', 0.026333778868341085)
('CLN6', 0.051336140863668125)
('CMTM4', 0.026459966972549133)
('COPS4', 0.011178157519620941)
('CRELD2', 0.02706288420574135)
('DCTN5', 0.015155215155215155)
('DNASE2', 0.014126927619273652)
('DSP', 0.004177109440267335)
('EID3', 0.0092057535959975)
('FAM72D', 0.004869316147511635)
('FEN1', 0.004810698994563914)
('FGFR1', 0.02550607287449393)
('GNA12', 0.00895296402356736)
('GPAA1', 0.009628403791183996)
('GTPBP4', 0.0268702

/Users/titti/opt/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
